# BIAFlows

## Nuclei Tracking 2D+t

### Fiji-workflow

The workflow treats the time-dimension as z-dimension and does a 3D-segmentation of the objects. The resulting object slices are then reduced to a center point.

#### Initialize the cytomine client and job

The cytomine client needs some parameters to connect to the server. The host on which the server is running, the public and private keys to make a connection, the id of the project and the id of the software.
The remaining 3 parameters will be passed on to the image analysis workflow.

In [ ]:
from getpass import getpass 

publicKey = getpass("Please enter the public key: ")
privateKey = getpass("Please enter the private key: ")

argv = ['--cytomine_public_key', publicKey, 
 '--cytomine_host',  'neubias.cytomine.be',
 '--cytomine_private_key',  privateKey,
 '--cytomine_id_project',  '1695226', 
 '--cytomine_id_software', '3804897',
 '--ij_gauss_radius',  '3',
 '--ij_threshold',  '60',
 '--ij_open_radius', '7']

Import CytomineJob and Job and update the status information.

In [ ]:
from cytomine import CytomineJob
from cytomine.models import Job
from beakerx.object import beakerx
jobID=-666
with CytomineJob.from_cli(argv) as cj:
        cj.job.update(status=Job.RUNNING, progress=0, statusComment="Initialisation...")
        jobID = cj.job.id
        beakerx.ij_gauss_radius = cj.parameters.ij_gauss_radius
        beakerx.ij_threshold = cj.parameters.ij_threshold
        beakerx.ij_open_radius = cj.parameters.ij_open_radius

#### Create local directories

Create the local working directories in a subfolder jobID of the user's home folder.

- in: input images
- out: output images
- ground_truth: ground truth images
- tmp: temporary path

In [ ]:
import os
from beakerx.object import beakerx
base_path = "{}".format(os.getenv("HOME"))
gt_suffix = "_lbl"
working_path = os.path.join(base_path, str(jobID))
in_path = os.path.join(working_path, "in")
out_path = os.path.join(working_path, "out")
gt_path = os.path.join(working_path, "ground_truth")
tmp_path = os.path.join(working_path, "tmp")

if not os.path.exists(working_path):
    os.makedirs(working_path)
    os.makedirs(in_path)
    os.makedirs(out_path)
    os.makedirs(gt_path)
    os.makedirs(tmp_path)

beakerx.in_path = in_path
beakerx.out_path = out_path

#### Download the images

In [ ]:
from cytomine.models import ImageGroupCollection
cj.job.update(progress=10, statusComment="Downloading images (to {})...".format(in_path))
image_group = ImageGroupCollection().fetch_with_filter("project", cj.parameters.cytomine_id_project)

input_images = [i for i in image_group if gt_suffix not in i.name]
gt_images = [i for i in image_group if gt_suffix in i.name]

for input_image in input_images:
    input_image.download(os.path.join(in_path, "{id}.tif"))

for gt_image in gt_images:
    related_name = gt_image.name.replace(gt_suffix, '')
    related_image = [i for i in input_images if related_name == i.name]
    if len(related_image) == 1:
        gt_image.download(os.path.join(gt_path, "{}.tif".format(related_image[0].id)))

#### Call the image analysis workflow 

In [ ]:
from subprocess import call

cj.job.update(progress=25, statusComment="Launching workflow...")
command = "/usr/bin/xvfb-run ./ImageJ-linux64 -macro NucleiTracking.ijm " \
		"\"input={}, output={}, gauss_rad={}, threshold={}, open_rad={}\" -batch".format(in_path, out_path, cj.parameters.ij_gauss_radius, cj.parameters.ij_threshold, cj.parameters.ij_open_radius)

return_code = call(command, shell=True, cwd="./Fiji.app")  # waits for the subprocess to return

if return_code != 0:
    err_desc = "Failed to execute the ImageJ macro (return code: {})".format(return_code)
    cj.job.update(progress=50, statusComment=err_desc)
    raise ValueError(err_desc)
cj.job.update(progress=30, statusComment="Workflow finished...")


#### Create and upload annotations

In [ ]:
from neubiaswg5.exporter.mask_to_points import mask_to_points_3d
import io
from skimage import io
from cytomine.models import Annotation,AnnotationCollection, ImageGroupCollection, ImageInstance, ImageSequenceCollection
from itertools import groupby
from shapely.geometry import LineString, Point
from shapely.affinity import affine_transform
import numpy as np
   
for image in cj.monitor(input_images, start=60, end=80, period=0.1, prefix="Extracting and uploading tracks from point-masks"):
    file = "{}.tif".format(image.id)
    path = os.path.join(out_path, file)
    data = io.imread(path)

    # extract objects
    objects = mask_to_points_3d(data, time=True)

    objects = sorted(objects, key=lambda annotationSlice: annotationSlice.time)    
    objects = sorted(objects, key=lambda annotationSlice: annotationSlice.label)    
    grouped = groupby(objects,  key=lambda annotationSlice: annotationSlice.label)
    
    image_sequences = ImageSequenceCollection().fetch_with_filter("imagegroup", image.id)
    
    depth_to_image = {iseq.time: iseq.image for iseq in image_sequences}
    height = ImageInstance().fetch(image_sequences[0].image).height
    
    collection = AnnotationCollection()
    
    groups = []
    uniquekeys = []
    for k, g in grouped:
        groups.append(list(g))      # Store group iterator as a list
        uniquekeys.append(k)
    
    for key, group in zip(uniquekeys, groups):
        listOfPoints = []
        for oSlice in group:
            listOfPoints.append((oSlice.polygon.x, oSlice.polygon.y))
        shape = Point(listOfPoints[0])
#       shape = LineString(listOfPoints)
        for oSlice2 in group:
            polygon = affine_transform(shape, [1, 0, 0, -1, 0, height])
            annotation = Annotation(location=polygon.wkt, id_image=depth_to_image[oSlice2.time], id_project=cj.parameters.cytomine_id_project, property=[{"key": "index", "value": str(oSlice2.label)}])
            collection.append(annotation)
        
    collection.save()
    print("Found {} objects in this image {}.".format(len(objects), image.id))
    
    

#### Calculate and upload metrics

In [ ]:
from neubiaswg5.metrics import computemetrics_batch
from cytomine.models import Property

cj.job.update(progress=80, statusComment="Computing and uploading metrics...")
outfiles, reffiles = zip(*[
    (os.path.join(out_path, "{}.tif".format(image.id)),
     os.path.join(gt_path, "{}.tif".format(image.id)))
    for image in input_images
])

results = computemetrics_batch(outfiles, reffiles, "PrtTrk", tmp_path)

for key, value in results.items():
    Property(cj.job, key=key, value=str(value)).save()
Property(cj.job, key="IMAGE_INSTANCES", value=str([im.id for im in input_images])).save()

#### Cleanup - remove the files and folders that have been downloaded or created.

In [ ]:
for image in input_images:
    file = str(image.id)
    path = out_path + "/" + file + ".tif"
    os.remove(path);
    path = in_path + "/" + file  + ".tif"
    os.remove(path);
    path = gt_path + "/" + file  + ".tif"
    os.remove(path);
    
tmpData = ['intracks.xml', 'intracks.xml.score.txt', 'reftracks.xml']
for file in tmpData:
    path = tmp_path + "/" + file
    os.remove(path);
if os.path.exists(working_path):
    os.rmdir(in_path)
    os.rmdir(out_path)
    os.rmdir(gt_path)
    os.rmdir(tmp_path)
    os.rmdir(working_path)

#### End the job and the connection to cytomine

In [ ]:
cj.job.update(status=Job.TERMINATED, progress=100, statusComment="Finished.")
print("END.")